 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'adult_bttwd.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-11-30 16:03:59】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\adult_bttwd.yaml
【INFO】【2025-11-30 16:04:04】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-11-30 16:04:04】【配置-数据】数据集=adult, k折=5, 目标列=income, 正类=">50K"
【INFO】【2025-11-30 16:04:04】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-30 16:04:04】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-11-30 16:04:04】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（延误/不延误）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='延误 vs 未延误比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始 DepDelay 的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是 "DepDelay"
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Airlines 原始数据加载与基本统计完成。')


【INFO】【2025-11-30 16:04:04】【数据加载完毕】样本数=32561，特征数=14，正类比例=0.24
【INFO】【2025-11-30 16:04:05】【数据集信息】名称=adult，样本数=32561，目标列=income，正类比例=24.08%


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


用于建模的标签列: income
原始目标列: income
count    32561.000000
mean         0.240810
std          0.427581
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: income, dtype: float64
【INFO】【2025-11-30 16:04:05】【步骤2摘要】Airlines 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-11-30 16:04:05】【预处理】连续特征=6个，类别特征=8个
【INFO】【2025-11-30 16:04:05】【预处理】编码后维度=100
【INFO】【2025-11-30 16:04:05】【预处理】编码特征维度=100，样本数=32561
【INFO】【2025-11-30 16:04:05】【步骤3摘要】特征预处理完成：连续=6，类别=8，编码维度=100。


In [5]:
# 步骤4：构建桶树并检查划分
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=df_raw.drop(columns=[cfg['DATA']['target_col']]).columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(df_raw.drop(columns=[cfg['DATA']['target_col']]))
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(lambda s: (s == cfg['DATA']['positive_label']).mean()).values
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')

【INFO】【2025-11-30 16:04:05】【桶树】已为样本生成桶ID，共 48 个组合


,bucket_id,count,pos_rate
0,L1_age=old|L2_education=high|L3_hours-per-week...,3211,0.0
1,L1_age=old|L2_education=mid|L3_hours-per-week=...,2713,0.0
2,L1_age=mid|L2_education=high|L3_hours-per-week...,2584,0.0
3,L1_age=mid|L2_education=mid|L3_hours-per-week=...,2321,0.0
4,L1_age=very_old|L2_education=mid|L3_hours-per-...,1654,0.0


【INFO】【2025-11-30 16:04:06】【步骤4摘要】桶树划分完成，共有 48 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-11-30 16:04:06】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-11-30 16:04:06】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
import numpy as np
import pandas as pd

print("y 全局标签分布：", np.unique(y, return_counts=True))

print("原始 income 列分布：")
print(df_raw[cfg['DATA']['target_col']].value_counts())


y 全局标签分布： (array([0]), array([32561], dtype=int64))
原始 income 列分布：
income
0    24720
1     7841
Name: count, dtype: int64


In [8]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-11-30 16:04:06】【基线-LogReg】使用决策阈值=0.400（fixed 模式）
【INFO】【2025-11-30 16:04:06】【基线-LogReg】第 1 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-LogReg】第 2 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-LogReg】第 3 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-LogReg】第 4 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-LogReg】第 5 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-LogReg】整体指标：
【INFO】【2025-11-30 16:04:06】【基线-RF】使用决策阈值=0.400（fixed 模式）
【INFO】【2025-11-30 16:04:06】【基线-RF】第 1 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-RF】第 2 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-RF】第 3 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-RF】第 4 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-RF】第 5 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-RF】整体指标：
【INFO】【2025-11-30 16:04:06】【基线-KNN】使用决策阈值=0.400（fixed 模式）
【INFO】【2025-11-30 16:04:06】【基线-KNN】第 1 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-KNN】第 2 折训练集仅包含单一类别，跳过该折
【INFO】【2025-11-30 16:04:06】【基线-KNN】第 3 折训练集仅包含单一类别，跳过该折
【I

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [16:04:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-30 16:04:07】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-30 16:04:10】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-30 16:04:10】【阈值】桶 ROOT 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=mid 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=old 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=very_old 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=young 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=others 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=mid|L2_education=high 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=mid|L2_education=low 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=mid|L2_education=mid 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=mid|L2_education=top 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=mid|L2_education=others 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_age=old|L2_education=high 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:10】【阈值】桶 L1_

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [16:04:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-30 16:04:12】【BTTWD】三支指标(含后悔)：Precision=0.000, Recall=0.000, F1=0.000, BAC=1.000, AUC=nan, MCC=0.000, Kappa=nan, BND_ratio=0.000, POS_Coverage=0.000, Regret=0.000
【INFO】【2025-11-30 16:04:12】【桶树】已为样本生成桶ID，共 47 个组合
【INFO】【2025-11-30 16:04:12】【K折实验】正在执行第 2/5 折...
【INFO】【2025-11-30 16:04:12】[BT] 使用桶评分配置：mode=bac, bac_weight=1.0, regret_weight=1.0, regret_sign=-1.0
【INFO】【2025-11-30 16:04:12】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-30 16:04:15】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-30 16:04:15】【阈值】桶 ROOT 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=mid 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=old 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=very_old 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=young 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=others 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=mid|L2_education=high 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:15】【阈值】桶 L1_age=mid|L2_e

...

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [16:04:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-30 16:04:28】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-30 16:04:31】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-30 16:04:31】【阈值】桶 ROOT 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=mid 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=old 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=very_old 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=young 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=others 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=mid|L2_education=high 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=mid|L2_education=low 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=mid|L2_education=mid 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=mid|L2_education=top 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=mid|L2_education=others 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_age=old|L2_education=high 标记为弱桶，阈值回退父桶/全局
【INFO】【2025-11-30 16:04:31】【阈值】桶 L1_

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


【INFO】【2025-11-30 16:04:33】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [9]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,n_all,pos_rate_all,parent_with_threshold,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,ROOT,L1,NaN,3645,1561,0.0,0.0,0.35,0.05,NaN,...,26048,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
1,L1_age=old,L1,ROOT,1248,601,0.0,0.0,0.35,0.05,NaN,...,8936,0.0,ROOT,NaN,NaN,NaN,NaN,NaN,1,0.0
2,L1_age=mid,L1,ROOT,958,451,0.0,0.0,0.35,0.05,NaN,...,6856,0.0,ROOT,NaN,NaN,NaN,NaN,NaN,1,0.0
3,L1_age=very_old,L1,ROOT,724,301,0.0,0.0,0.35,0.05,NaN,...,5136,0.0,ROOT,NaN,NaN,NaN,NaN,NaN,1,0.0
4,L1_age=young,L1,ROOT,799,352,0.0,0.0,0.35,0.05,NaN,...,5120,0.0,ROOT,NaN,NaN,NaN,NaN,NaN,1,0.0


C:\Users\xzq20.HUANLI\AppData\Local\Temp\ipykernel_19092\946468196.py:9: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


【INFO】【2025-11-30 16:04:38】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [10]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。')

【INFO】【2025-11-30 16:04:38】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'bucket_thresholds_per_fold.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_kfold_summary_xgb_twd.csv', 'metrics_overview.csv', 'unsw_nb15']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-30 16:04:38】【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。
